In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score, average_precision_score, classification_report, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = pd.read_csv('../Data/heart_disease_uci.csv')

In [ ]:
df = df.dropna()

df = pd.get_dummies(df, drop_first=True)

X_reg = df.drop(columns=['chol'])  
y_reg = df['chol']

X_clf = df.drop(columns=['num'])   
y_clf = (df['num'] > 0).astype(int)